In [ ]:
!pip install DataLoader
!pip install transformers
!pip install pytorch-crf
!pip install datasets
!pip install RobertaModel
!pip install seqeval
!pip install datasets
!pip install datasets transformers[sentencepiece]

ERROR: Could not find a version that satisfies the requirement RobertaModel (from versions: none)
ERROR: No matching distribution found for RobertaModel


In [ ]:
from datasets import load_dataset
import json
from transformers import AutoModel
import torch
from torchcrf import CRF
import torch.nn as nn
from seqeval.metrics import classification_report
from datasets import DatasetDict, Dataset
import numpy as np
from datasets import DatasetDict, Dataset
from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader
from transformers import DataCollatorForTokenClassification

# NeRUBioS dataset
nerubios_dataset_train = load_dataset("json", data_files="/content/NeRUBioS_train.json", field="data")
nerubios_dataset_dev = load_dataset("json", data_files="/content/NeRUBioS_dev.json", field="data")
nerubios_dataset_test = load_dataset("json", data_files="/content/NeRUBioS_test.json", field="data")

In [ ]:
nerubios_dataset_dev

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 1840
    })
})

In [ ]:
import numpy as np
from datasets import DatasetDict, Dataset

nerubios_dataset_train = DatasetDict({
    "train": Dataset.from_dict({
        "tokens": nerubios_dataset_train["train"]["tokens"],
        "ner_tags": [
            [0 if tag in [ 7, 8, 9, 10] else tag for tag in example]
            for example in nerubios_dataset_train["train"]["ner_tags"]
        ]
    })
})

nerubios_dataset_test = DatasetDict({
    "train": Dataset.from_dict({
        "tokens": nerubios_dataset_test["train"]["tokens"],
        "ner_tags": [
            [0 if tag in [ 7, 8, 9, 10] else tag for tag in example]
            for example in nerubios_dataset_test["train"]["ner_tags"]
        ]
    })
})



nerubios_dataset_dev = DatasetDict({
    "train": Dataset.from_dict({
        "tokens": nerubios_dataset_dev["train"]["tokens"],
        "ner_tags": [
            [0 if tag in [ 7, 8, 9, 10] else tag for tag in example]
            for example in nerubios_dataset_dev["train"]["ner_tags"]
        ]
    })
})


from datasets import DatasetDict
#For training, development, and testing partitions
raw_datasets = DatasetDict({
    'train': nerubios_dataset_train['train'],
    'validation': nerubios_dataset_dev['train'],
    'test': nerubios_dataset_test['train']
    })
label_names = ["O", "B-NegREF", "I-NegREF", "B-NEG", "I-NEG", "B-NSCO", "I-NSCO"]


In [ ]:
from transformers import AutoTokenizer
import datasets

model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"  # Mejor modelo

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
tokenizer.is_fast

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -1 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-1)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


def detokenize(tokens):
    # Convierte los tokens en una cadena de texto
    text = tokenizer.decode(tokens, skip_special_tokens=True)
    return text

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/13832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1840 [00:00<?, ? examples/s]

Map:   0%|          | 0/2765 [00:00<?, ? examples/s]

In [ ]:
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# Paso 1: Definir la arquitectura del modelo NER
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
config = RobertaConfig.from_pretrained(model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint, config=config)
num_labels = 7


# Definir la arquitectura del modelo CRF
class NERModel(nn.Module):
    def __init__(self, model, num_labels):
        super(NERModel, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.crf = CRF(num_labels)  # Agregar capa CRF

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        return logits

    def loss(self, logits, labels, attention_mask):
        mask = attention_mask.bool()  # Convertir la máscara a tipo booleano
        loss = -self.crf(logits, labels, mask=mask, reduction='mean')
        return loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ner_model = NERModel(model,num_labels).to(device)
# Configuración del modelo y los datos
config = tokenizer.model_max_length
num_labels = 9  # Agregar tokens especiales para CRF


# Crear dataloaders
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_ids"]), reverse=True)
    input_ids = [torch.tensor(example["input_ids"]) for example in batch]
    attention_mask = [torch.tensor(example["attention_mask"]) for example in batch]
    labels = [torch.tensor(example["labels"]) for example in batch]
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-1)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

dataloader_train = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True, collate_fn=collate_fn)

# Crear modelo CRF

optimizer = torch.optim.AdamW(ner_model.parameters(), lr=1e-5)

# Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    ner_model.train()
    for batch in dataloader_train:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = ner_model(input_ids, attention_mask)
        loss = ner_model.loss(logits, labels, attention_mask)  # Utilizar la función de pérdida CRF
        loss.backward()
        optimizer.step()

    torch.cuda.empty_cache()  # Liberar memoria de la GPU

    print(f"Epoch {epoch+1}")

Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10


In [59]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
torch.save(ner_model.state_dict(), "/content/drive/MyDrive/VIU/MODELO/ner_model.pth")

In [61]:
dataloader_test = DataLoader(tokenized_datasets["test"], batch_size=16, shuffle=True, collate_fn=collate_fn)

In [ ]:
from seqeval.metrics import classification_report

def convert_labels_to_text(labels):
    return [label_names[label] for label in labels]


# Función para evaluar el modelo
def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask)
            predicted_labels = torch.argmax(logits, dim=2)

            for i in range(len(predicted_labels)):
                predicted = predicted_labels[i]
                true_labels = labels[i]

                # Ignorar las etiquetas PAD (-100)
                mask = true_labels != -100
                predicted = predicted[mask]
                true_labels = true_labels[mask]

                all_predictions.append(predicted)
                all_labels.append(true_labels)

    all_predictions_text = [convert_labels_to_text(labels) for labels in all_predictions]
    all_labels_text = [convert_labels_to_text(labels) for labels in all_labels]

    report = classification_report(all_labels_text, all_predictions_text, output_dict=True)
    filtered_report = {label: report[label] for label in report.keys() if label in ["NEG", "NSCO"]}

    filtered_report_str = "              precision    recall  f1-score   support\n\n"
    for label, metrics in filtered_report.items():
        filtered_report_str += f"   {label:7}   {metrics['precision']:.2f}      {metrics['recall']:.2f}      {metrics['f1-score']:.2f}      {metrics['support']}\n"

    return filtered_report_str

# Evaluar el modelo en el conjunto de prueba
report = evaluate_model(ner_model, dataloader_test)
print("Informe de clasificación:")
print(report)
